In [1]:
from google.cloud import bigquery
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as pyplot

import warnings 
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
client = bigquery.Client()

In [3]:
data_ref = client.dataset("Cardio", project="learnbigquerry")
dataset = client.get_dataset(data_ref)
table = dataset.table("Cardio_train")

In [4]:
query = """
SELECT * 
FROM learnbigquerry.Cardio.Cardio_train
 """

query_job = client.query(query)
results = query_job.to_dataframe()

results.to_csv("Cardio.csv", index=False)

In [5]:
df  = pd.read_csv("Cardio.csv", delimiter=";")
df

C:\Users\USER\AppData\Local\Temp\ipykernel_6432\3562787214.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df  = pd.read_csv("Cardio.csv", delimiter=";")


,,,,,,,,,,,,string_field_0
id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [6]:

dfo = df
dfo.to_csv("Nc.csv", index=True, encoding='utf-8')

In [7]:
columns_names = ["Id", "Age", "Gender", "Height", "Weight", "Systolic", "Diastolic",
                  "Cholesterol", "Glucose", "Smoking", "Alchohol", "Active", "Cardio"]
cardio = pd.read_csv("Nc.csv", names=columns_names)

C:\Users\USER\AppData\Local\Temp\ipykernel_6432\1619205801.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  cardio = pd.read_csv("Nc.csv", names=columns_names)


In [8]:
cardio.head(5)

,Id,Age,Gender,Height,Weight,Systolic,Diastolic,Cholesterol,Glucose,Smoking,Alchohol,Active,Cardio
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,string_field_0
1,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
2,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
3,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
4,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


In [9]:
cardio = cardio.drop([0, 1], axis=0)

In [10]:
cardio = cardio.set_index("Id")


In [11]:
# to convert Age in days to years
cardio["Age"]= cardio["Age"].astype(float)
cardio["Age"] = cardio["Age"]/365.25
cardio["Age"] = cardio.Age.round().astype(int)

In [12]:
# Converting Height to Meters
cardio["Height"] = cardio["Height"].astype(int)
cardio["Height in Meters"] = cardio["Height"] / 100

In [13]:
# to calculate and store BMI 
cardio["Weight"] = cardio["Weight"].astype(float)
cardio["BMI"] = cardio["Weight"] / (cardio["Height in Meters"] * cardio["Height in Meters"])

In [14]:
cardio["Systolic"], cardio["Diastolic"] = cardio["Systolic"].astype(int), cardio["Diastolic"].astype(int)

In [15]:
cardio["Cardio"], cardio["Active"] = cardio["Cardio"].astype(int), cardio["Active"].astype(int)

In [16]:
cardio["Smoking"], cardio["Gender"] = cardio["Smoking"].astype(int), cardio["Gender"].astype(int)
cardio["Glucose"], cardio["Alchohol"] = cardio["Glucose"].astype(int), cardio["Alchohol"].astype(int)
cardio["Cholesterol"]= cardio["Cholesterol"].astype(int)

In [17]:
cardio.dropna(inplace=True)
cardio.drop_duplicates(inplace=True)

In [18]:
# to reset index after droping null entries and duplicates. 
cardio.reset_index(inplace=True)

In [19]:
cardio.describe()

,Age,Gender,Height,Weight,Systolic,Diastolic,Cholesterol,Glucose,Smoking,Alchohol,Active,Cardio,Height in Meters,BMI
count,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000,66179.000000
mean,53.327445,1.357107,164.345684,74.581171,129.306850,97.609332,1.386014,1.238142,0.092748,0.056710,0.797549,0.513713,1.643457,27.704164
std,6.803794,0.479150,8.353594,14.613557,158.374251,193.789748,0.692225,0.584110,0.290082,0.231289,0.401830,0.499816,0.083536,6.201824
min,30.000000,1.000000,55.000000,10.000000,-150.000000,-70.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.550000,3.471784
25%,48.000000,1.000000,159.000000,65.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.590000,23.875433
50%,54.000000,1.000000,165.000000,72.000000,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.650000,26.573129
75%,58.000000,2.000000,170.000000,83.000000,140.000000,90.000000,2.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.700000,30.470522
max,65.000000,2.000000,250.000000,200.000000,16020.000000,11000.000000,3.000000,3.000000,1.000000,1.000000,1.000000,1.000000,2.500000,298.666667


In [20]:
cardio.dtypes

Id                   object
Age                   int32
Gender                int32
Height                int32
Weight              float64
Systolic              int32
Diastolic             int32
Cholesterol           int32
Glucose               int32
Smoking               int32
Alchohol              int32
Active                int32
Cardio                int32
Height in Meters    float64
BMI                 float64
dtype: object

In [21]:
cardio = cardio[cardio["Weight"] >= 25]

In [23]:
cardio.reset_index(inplace=True)

In [24]:
cardio.to_csv("Cleaned.csv",index=False)